In [28]:
# 以两层网络为例，进行算法的实现
# 实现一个两层网络的类
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)

def softmax(x):
    x = x - np.max(x, axis=-1, keepdims=True)
    return np.exp(x) / np.sum(np.exp(x), axis=-1, keepdims=True)

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val
        it.iternext()   
        
    return grad

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size,weight_init_std=0.01):
        # 初始化权重
        # 该网络为两层结构，每层都包含权重和偏置
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    def predict(self, x):
        # 网络的前向过程，但是输出为预测的类
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        return y
    # x:输入数据, t:监督数据
    def loss(self, x, t):
        # 网络的前向过程，输出为损失函数的值
        y = self.predict(x)
        return cross_entropy_error(y, t)
    def accuracy(self, x, t):
        # 计算准确率
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    # x:输入数据, t:监督数据
    # 注意！这里不是对x进行求导，而是对权重w进行求导！
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    def gradient(self, x, t):
        # 高速计算方法
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        dz1 = np.dot(dy, W2.T)
        da1 = sigmoid_grad(a1) * dz1
        grads['W1'] = np.dot(x.T, da1)
        grads['b1'] = np.sum(da1, axis=0)

        return grads

In [29]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
print(net.params['W1'].shape) # (784, 100)
print(net.params['b1'].shape) # (100,)
print(net.params['W2'].shape) # (100, 10)
print(net.params['b2'].shape) # (10,)

(784, 100)
(100,)
(100, 10)
(10,)


In [30]:
x = np.random.rand(100, 784) # 伪输入数据（100笔）
y = net.predict(x)
t = np.random.rand(100, 10) # 伪正确解标签（100笔）
# grads = net.numerical_gradient(x, t) # 计算梯度
# 计算一次需要1分钟多
# 说明该处的计算方法效率很低
# grads = net.gradient(x,t)

In [31]:
# grads1 = net.numerical_gradient(x, t) # 计算梯度
grads2 = net.gradient(x,t)

In [32]:
# print(grads1["W1"])

In [33]:
print(grads2["W1"])

[[ 1.34752027e-03  1.82125671e-03 -1.31617800e-03 ... -5.28669867e-04
   1.01482046e-03 -8.05772359e-05]
 [ 1.04284062e-03  1.54202210e-03 -1.03392453e-03 ... -4.95441525e-04
   7.47054180e-04 -7.19297295e-05]
 [ 1.05692534e-03  1.44323646e-03 -9.62270635e-04 ... -3.80147109e-04
   8.84357287e-04 -4.78915992e-05]
 ...
 [ 1.00577589e-03  1.61772564e-03 -1.03714972e-03 ... -5.73434768e-04
   6.85140632e-04  3.60114022e-05]
 [ 1.13364962e-03  1.81958958e-03 -1.23044050e-03 ... -6.07022532e-04
   7.58019186e-04  1.12209276e-05]
 [ 1.33219627e-03  1.68396372e-03 -1.10228766e-03 ... -4.35390079e-04
   1.01793246e-03 -7.82100888e-05]]


In [34]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 计算梯度
    #grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)
    
    # 利用梯度更新参数
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

train acc, test acc | 0.0993, 0.1032
train acc, test acc | 0.7773666666666667, 0.7846
train acc, test acc | 0.87625, 0.8796
train acc, test acc | 0.8962666666666667, 0.8999
train acc, test acc | 0.9070333333333334, 0.91
train acc, test acc | 0.9134, 0.9153
train acc, test acc | 0.9192, 0.922
train acc, test acc | 0.9228166666666666, 0.9242
train acc, test acc | 0.9271833333333334, 0.9289
train acc, test acc | 0.93015, 0.9293
train acc, test acc | 0.9336833333333333, 0.9324
train acc, test acc | 0.9370833333333334, 0.9361
train acc, test acc | 0.93865, 0.939
train acc, test acc | 0.94145, 0.9399
train acc, test acc | 0.9437666666666666, 0.9428
train acc, test acc | 0.9452833333333334, 0.9431
train acc, test acc | 0.9477, 0.9471
